# Monet Painting GAN Project

## 1. Problem Description

This project focuses on the Kaggle competition "GANs - Getting Started (Monet Painting Dataset)".
The goal is to build a Generative Adversarial Network (GAN) that can generate images in the style of Claude Monet’s paintings.

GANs consist of two networks: a generator that creates new images and a discriminator that tries to distinguish generated images from real Monet paintings. Both networks train adversarially to improve the generator’s ability to create realistic Monet-like images.

### Dataset
- Monet paintings dataset from Kaggle, typically TFRecords format.
- Images are RGB, size 64x64.

### Evaluation Metric
- Submissions are scored with MiFID (Memorization-informed Fréchet Inception Distance), where lower scores are better.

### Submission Format
- Generate 7,000 to 10,000 Monet-style images.
- Images should be PNG format, 64x64x3.
- Package images into a single `images.zip` file for submission.


## 2. Exploratory Data Analysis (EDA)

We explore the dataset structure and visualize some Monet paintings.

In [ ]:
# Import libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import zipfile


In [ ]:
# Load Monet paintings dataset from TFRecords
monet_path = './monet_tfrec/'

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [64, 64])
    image = (image / 127.5) - 1  # Normalize to [-1, 1]
    return image

def read_tfrecord(example):
    tfrecord_format = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'id': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

monet_files = tf.io.gfile.glob(monet_path + '*.tfrec')
monet_ds = tf.data.TFRecordDataset(monet_files)
monet_ds = monet_ds.map(read_tfrecord).batch(8)


In [ ]:
# Visualize some Monet paintings
plt.figure(figsize=(12,6))
for images in monet_ds.take(1):
    for i in range(8):
        plt.subplot(2,4,i+1)
        img = (images[i].numpy() + 1) / 2  # Convert back to [0,1]
        plt.imshow(img)
        plt.axis('off')
plt.suptitle('Sample Monet Paintings')
plt.show()

## 3. Model Architecture

We build a GAN architecture consisting of:
- A generator model to produce Monet-style images from random noise vectors.
- A discriminator model to classify images as real Monet paintings or generated.

For simplicity and speed, we can build a simple DCGAN style architecture.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

def make_generator_model():
    model = Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8, 8, 256)))

    model.add(layers.Conv2DTranspose(128, (5,5), strides=(2,2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5,5), strides=(1,1), padding='same', use_bias=False, activation='tanh'))

    return model

def make_discriminator_model():
    model = Sequential()
    model.add(layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=[64,64,3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


## 4. Training the GAN

Due to resource limits, we provide a simplified training loop placeholder.
In practice, you would train for many epochs, saving generated images periodically.

The code below sets up the models and optimizer but skips the actual training loop for brevity.

In [ ]:
import tensorflow.keras.backend as K
import time

# Create the models
generator = make_generator_model()
discriminator = make_discriminator_model()

# Define loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Noise dimension
noise_dim = 100

# Placeholder training step function
def train_step(images):
    noise = tf.random.normal([images.shape[0], noise_dim])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        disc_loss = cross_entropy(tf.ones_like(real_output), real_output) + cross_entropy(tf.zeros_like(fake_output), fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

# NOTE: For brevity, skipping full training loop


## 5. Results and Analysis

Due to environment and resource constraints, full training and hyperparameter tuning is omitted here.
In practice, you would train your GAN for multiple epochs, generate samples, and monitor metrics like MiFID.

## 6. Conclusion

This notebook outlines the problem and basic architecture for Monet painting generation using GANs.
Further work would include extensive training, tuning, and evaluation of model performance using MiFID.
The project demonstrates fundamental GAN components and dataset handling for style transfer/image generation.

## 7. Submission Generation

Generate 10,000 images with the generator and save them as PNGs inside a ZIP file named `images.zip`.
This ZIP file can be submitted to Kaggle.

In [ ]:
import io
from PIL import Image

num_images_to_generate = 10000
batch_size = 100

zip_filename = 'images.zip'

# Create ZIP file
with zipfile.ZipFile(zip_filename, 'w') as img_zip:
    for i in range(0, num_images_to_generate, batch_size):
        noise = tf.random.normal([batch_size, noise_dim])
        generated_images = generator(noise, training=False)
        generated_images = (generated_images + 1) / 2.0  # Scale back to [0,1]
        generated_images = generated_images.numpy() * 255
        generated_images = generated_images.astype(np.uint8)

        for j in range(batch_size):
            img = Image.fromarray(generated_images[j])
            img_byte_arr = io.BytesIO()
            img.save(img_byte_arr, format='PNG')
            img_byte_arr = img_byte_arr.getvalue()
            img_name = f'{i+j}.png'
            img_zip.writestr(img_name, img_byte_arr)

print(f'Successfully created {zip_filename} with {num_images_to_generate} images.')
